The UCLA miniscope system produces unqiue timestamp logs for each miniscope and behavioral camera used in a session. This script:

1) concatenates timestamp logs from the same device (if you generated multiple logs per device, e.g., if you recorded multiple videos in the same session)

2) aligns frames from each device (and minian's output) to the closest unifying timestamps for feed synchronization

IMPORTANT: this script assumes that data directory is organized as .../session_date/trial_time/devices

### Step 0: load timestamp logs (either multiple or one per device per session)

In [2]:
import pandas as pd
import os
import re

#identify imaging session directory
sess_dir = "//penc2.rc.int.colorado.edu/DonaldsonLab/Sheeran/miniscope_files/wms/test_miniscope_preprocess_220710/4517/2022_07_09"
a = os.listdir(sess_dir)

#put all folders with HH_MM_SS format into list
trials = []

date_patt = "[0-2][0-9]_[0-5][0-9]_[0-5][0-9]"

for i in a:
    if re.search(date_patt, i):
        trials.append(i)

#place in chronological order
trials.sort()

In [3]:
#check that your trials were correctly ordered temporally and change if not
print(trials)

['17_19_13', '17_26_00', '17_32_11', '17_38_55', '17_44_59', '17_51_46', '17_58_31']


In [15]:
#for loop that goes into each folder in trials, opens the MiniCam1, MiniCam2, and My_V4_Miniscope folders 
#and grabs the timestamp .csv from each

ms1 = "My_V4_Miniscope" #may need to change this with multi-scope setup
#ms2 = "My_V4_Miniscope2" #i don't actually yet know what the nomenclature will be for a second scope
mc1 = "MiniCam1"
mc2 = "MiniCam2"

devices = [ms1, mc1, mc2] #ms2

ts_files = []


ms1_ts_files = []
#ms2_ts_files = []
mc1_ts_files = []
mc2_ts_files = []

#can nest these for loops likely
for j in trials:
    ms1_ts_path = os.path.join(sess_dir, j, ms1, "timeStamps.csv").replace("\\","/")
    ms1_ts_files.append(ms1_ts_path)
    
#for k in trials:
#    ms2_ts_path = os.path.join(sess_dir, k, ms2, "timeStamps.csv").replace("\\","/")
#    ms2_ts_files.append(ms2_ts_path)

for l in trials:
    mc1_ts_path = os.path.join(sess_dir, j, mc1, "timeStamps.csv").replace("\\","/")
    mc1_ts_files.append(mc1_ts_path)
    
for m in trials:
    mc2_ts_path = os.path.join(sess_dir, j, mc2, "timeStamps.csv").replace("\\","/")
    mc2_ts_files.append(mc2_ts_path)

### Step 1: concatenating each device's timestamp logs (not necessary if you only have one recording per device per session)

I assume the videoWriter function in the video concat script immediately places one videos' frames after the previous (ie frame 0 on second video becomes frame n+1 when n = last frame of first video). We should confirm this.

I also assume that Bento or other things importing concatenated files will want timestamps to be continuous integers throughout all the timestamps, not something like 1.123345, 2.12043034, where the prefix would denote which original file the timestamp came from

With those assumptions, the strategy in this step is to:

a) load first timestamp file from all streams (assuming your trials are ordered correctly, see above cell)

b) find maximum  frame # of each timestamp file (ms1_max = x, mc1_max = y, mc2_max = z) and save each as its own variable

c) find maximum timestamp of each timestamp file and compare to find the latest timestamp across the three streams' first video

d) save this max timestamp as its own variable and add 60000 (as a one minute spacer for a faux start time for the next video)

e) save each timestamp file into a new table

f) load next timestamp file for all streams

g) repeat b)

h) add the max frame from the previous file to each of the current frame numbers

i) repeat c) and d)

j) add the value calculated in d) to all timestamps in all loaded files

k) concatenate each timestamp file to previous file

l) repeat f-k for remaining files 

In [50]:
for p in range(len(trials)):
    #read ts files into dfs - any reason to make these a dictionary?
    mscope1 = pd.read_csv(ms1_ts_files[p])
    #mscope2 = pd.read_csv(ms2_ts_files[p])
    mcam1 = pd.read_csv(mc1_ts_files[p])
    mcam2 = pd.read_csv(mc2_ts_files[p])
    
    if p == 0:
        #find max frame of each
        ms1_maxF = mscope1['Frame Number'].max()
        mc1_maxF = mcam1['Frame Number'].max()
        mc2_maxF = mcam2['Frame Number'].max()
    
        #find max timestamp across all files, add 60000
        ms1_maxTS = mscope1['Time Stamp (ms)'].max()
        mc1_maxTS = mcam1['Time Stamp (ms)'].max()
        mc2_maxTS = mcam2['Time Stamp (ms)'].max()
        maxTSs = [ms1_maxTS, mc1_maxTS, mc2_maxTS]
        
        next_start_time = (max(maxTSs) + 60000)

        #compile dfs into overall dfs
        ms1_concat = mscope1
        #ms2_concat = mscope2
        mc1_concat = mcam1
        mc2_concat = mcam2
        
    else:
        #add the maxF from each feed type to the next column before repeating calculating the new maxF
        mscope1['Frame Number'] = mscope1['Frame Number'] + ms1_maxF
        mcam1['Frame Number'] = mcam1['Frame Number'] + mc1_maxF
        mcam2['Frame Number'] = mcam2['Frame Number'] + mc2_maxF
        
        ms1_maxF = mscope1['Frame Number'].max()
        mc1_maxF = mcam1['Frame Number'].max()
        mc2_maxF = mcam2['Frame Number'].max()
        
        #add the next start time to all feeds' timestamp column before repeating calculating next next start time
        mscope1['Time Stamp (ms)'] = mscope1['Time Stamp (ms)'] + next_start_time
        mcam1['Time Stamp (ms)'] = mcam1['Time Stamp (ms)'] + next_start_time
        mcam2['Time Stamp (ms)'] = mcam2['Time Stamp (ms)'] + next_start_time
        
        ms1_maxTS = mscope1['Time Stamp (ms)'].max()
        mc1_maxTS = mcam1['Time Stamp (ms)'].max()
        mc2_maxTS = mcam2['Time Stamp (ms)'].max()
        maxTSs = [ms1_maxTS, mc1_maxTS, mc2_maxTS]
        
        next_start_time = (max(maxTSs) + 60000)
        
        #concat modified dfs to overall dfs
        ms1_concat = pd.concat([ms1_concat,mscope1], axis = 0)
        mc1_concat = pd.concat([mc1_concat,mcam1], axis = 0)
        mc2_concat = pd.concat([mc2_concat,mcam2], axis = 0)
        

In [53]:
mc2_concat

,Frame Number,Time Stamp (ms),Buffer Index
0,0,-21,1
1,1,3,1
2,2,22,0
3,3,48,2
4,4,64,1
...,...,...,...
14311,100201,2494695,0
14312,100202,2494720,0
14313,100203,2494738,0
14314,100204,2494759,0


### Step 2: aligning frames from each device to closest unifying timestamp

In [ ]:
#unfinished - want to load minian C variable into this script and confirm its frames agree with the miniscope frames generated above
#miniscope files across trials compiled by rename_files matlab script and videos concatenated by minian
#minian's output lacks the time stamps but the frames should be identical to the miniscope frames if not downsampled

#load minian 
#note - figure out best way to get minian C variable into here - may be pending bento import scripts
#minian_C_path = xyz

minian_frames = np.asarray(minian.C.frame) #from Will Mau (Sinai)'s script

if minian_frames == ms1_concat['Frame Number']:
    print("Minian and miniscope_1 frames agree")
else:
    print("Minian and miniscope_1 frames disagree; check concatenation")
    

In [ ]:
#unfinished - need to figure out the application of pulling out various timestamps (custom code, what Bento wants, etc.)

#want to create a new column in the mc1 and mc2 concate tables that
#indicate the frame number of the best corresponding minian(/miniscope camera) frame, per the timestamps
    
    
#mc1_concat['ms1_frame'] =
#mc2_concat['ms2_frame'] = 


